In [15]:
using BenchmarkTools

# 文字列の分割（split）

In [6]:
line = "0"
for i in 1:10000
    line = line * ",$i"
end

In [9]:
@benchmark strs = split(line, ",")

BenchmarkTools.Trial: 
  memory estimate:  569.17 KiB
  allocs estimate:  10015
  --------------
  minimum time:     861.500 μs (0.00% GC)
  median time:      956.799 μs (0.00% GC)
  mean time:        1.178 ms (4.53% GC)
  maximum time:     32.675 ms (93.38% GC)
  --------------
  samples:          4225
  evals/sample:     1

# 配列の初期化

## push!で値を追加

### サイズ指定なし・型指定なし

In [17]:
@benchmark begin
    list = []
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  3.52 MiB
  allocs estimate:  99506
  --------------
  minimum time:     2.035 ms (0.00% GC)
  median time:      2.238 ms (0.00% GC)
  mean time:        2.790 ms (9.78% GC)
  maximum time:     20.212 ms (0.00% GC)
  --------------
  samples:          1788
  evals/sample:     1

### サイズ指定なし・型指定あり

In [16]:
@benchmark begin
    list = Vector{Int64}()
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  2.00 MiB
  allocs estimate:  17
  --------------
  minimum time:     949.800 μs (0.00% GC)
  median time:      1.005 ms (0.00% GC)
  mean time:        1.172 ms (0.00% GC)
  maximum time:     9.379 ms (0.00% GC)
  --------------
  samples:          3797
  evals/sample:     1

### サイズヒントあり・型指定あり

In [19]:
@benchmark begin
    list = Vector{Int64}()
    sizehint!(list, 100000)
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  781.39 KiB
  allocs estimate:  2
  --------------
  minimum time:     558.200 μs (0.00% GC)
  median time:      848.000 μs (0.00% GC)
  mean time:        920.887 μs (0.00% GC)
  maximum time:     10.502 ms (0.00% GC)
  --------------
  samples:          5028
  evals/sample:     1

## list[index] = value で値をセット

In [20]:
@benchmark begin
    list = Vector{Int64}(undef, 100000)
    for i in 1:100000
        list[i] = i
    end
end

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     75.400 μs (0.00% GC)
  median time:      94.201 μs (0.00% GC)
  mean time:        237.837 μs (13.86% GC)
  maximum time:     5.976 ms (56.88% GC)
  --------------
  samples:          10000
  evals/sample:     1

**push!でなく値をセットする方式が10倍速い（push!による初期化は遅い）**

## リスト内包表記による初期化

### 型指定なし

In [23]:
@benchmark list = [i for i in 1:100000]

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     42.201 μs (0.00% GC)
  median time:      87.600 μs (0.00% GC)
  mean time:        131.865 μs (9.79% GC)
  maximum time:     7.840 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

**リストの初期化はリスト内包表記が速い**

### 型指定あり

In [24]:
@benchmark list = Int64[i for i in 1:100000]

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     60.499 μs (0.00% GC)
  median time:      88.401 μs (0.00% GC)
  mean time:        116.101 μs (9.37% GC)
  maximum time:     7.253 ms (98.38% GC)
  --------------
  samples:          10000
  evals/sample:     1

## リストの初期化まとめ
- **リスト内包表記またはサイズ指定して値をセットする方式が速い**
- リスト内包表記の型指定は速度に影響しないかも（今回試したのがプリミティブだからかも）
- push!は遅いが便利なのでパフォーマンスを気にしない場面で用いる

# Dictの初期化

## push!による初期化

In [30]:
@benchmark begin
    dict = Dict{Int64,Int64}()
    for i in 1:100000
        push!(dict, i => i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  5.67 MiB
  allocs estimate:  36
  --------------
  minimum time:     4.983 ms (0.00% GC)
  median time:      5.806 ms (0.00% GC)
  mean time:        6.703 ms (8.89% GC)
  maximum time:     15.456 ms (36.36% GC)
  --------------
  samples:          744
  evals/sample:     1

## 辞書内包表記による初期化

In [31]:
@benchmark begin
    dict = Dict{Int64,Int64}(
        i => i for i in 1:100000
    )
end

BenchmarkTools.Trial: 
  memory estimate:  5.67 MiB
  allocs estimate:  36
  --------------
  minimum time:     4.997 ms (0.00% GC)
  median time:      5.599 ms (0.00% GC)
  mean time:        6.384 ms (9.17% GC)
  maximum time:     16.823 ms (0.00% GC)
  --------------
  samples:          781
  evals/sample:     1

## リストを作って初期化

### Dict(zip(keys, values))

In [32]:
@benchmark begin
    keys = Int64[i for i in 1:100000]
    values = Int64[i for i in 1:100000]
    dict = Dict(zip(keys,values))
end

BenchmarkTools.Trial: 
  memory estimate:  7.20 MiB
  allocs estimate:  42
  --------------
  minimum time:     6.327 ms (0.00% GC)
  median time:      7.151 ms (0.00% GC)
  mean time:        7.980 ms (7.90% GC)
  maximum time:     18.302 ms (39.85% GC)
  --------------
  samples:          625
  evals/sample:     1

### Dict(keys .=> values)

In [33]:
@benchmark begin
    keys = Int64[i for i in 1:100000]
    values = Int64[i for i in 1:100000]
    dict = Dict(keys .=> values)
end

BenchmarkTools.Trial: 
  memory estimate:  8.72 MiB
  allocs estimate:  42
  --------------
  minimum time:     6.370 ms (0.00% GC)
  median time:      7.459 ms (0.00% GC)
  mean time:        8.352 ms (10.15% GC)
  maximum time:     18.627 ms (0.00% GC)
  --------------
  samples:          598
  evals/sample:     1

## Dictの初期化まとめ
- **「push!」か「辞書内包表記」が速く、「リストにセットして辞書を初期化」はちょっと遅い**